In [1]:
import torch
import torch.nn as nn
import numpy as np
import snntorch as snn
import matplotlib.pyplot as plt
from snntorch import spikegen
from snntorch import surrogate
from snntorch import utils
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import tensorflow as tf
import random
import csv
from forest_fire_snn import SNN
import pandas as pd

In [ ]:
# Create an instance of SNN Object
net = SNN()

# Prepare CSV data
df = pd.read_csv('firetest_data.csv')
features = df[['Temp', 'Audio', 'Humidity', 'CO2']]
labels = df['Fire']

# Normalize features
df['Temp'] = df['Temp'] / 100
df['Humidity'] = df['Humidity'] / 100
df['CO2'] = df['CO2'] / 5000

X = df[['Temp', 'Audio', 'Humidity', 'CO2']]
Y = df['Fire'].to_numpy()

X_np = X.to_numpy()
Y_np = Y
X_tensor = torch.tensor(X_np, dtype=torch.float32)
Y_tensor = torch.tensor(Y_np, dtype=torch.long)

# Convert features to spike trains
num_steps = 25
def to_spike_trains(X_tensor, num_steps=25):
    num_samples, num_features = X_tensor.shape
    spike_data = torch.zeros((num_samples, num_steps, num_features))
    
    for i in range(num_samples):
        for t in range(num_steps):
            spike_data[i, t] = torch.bernoulli(X_tensor[i])
    return spike_data

fire_train = to_spike_trains(X_tensor, num_steps=num_steps)

class FireDataset(torch.utils.data.Dataset):
    def __init__(self, spike_data, labels):
        self.spike_data = spike_data
        self.labels = labels
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        return self.spike_data[idx], self.labels[idx]
    
dataset = FireDataset(fire_train, Y_tensor)
train_loader = torch.utils.data.DataLoader(dataset, batch_size=64, shuffle=True)

# Training Loop
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=5e-4)

num_epochs = 200

for epoch in range(num_epochs):
    net.train()
    total_loss, total_correct, total_n = 0.0, 0, 0
    
    for data, targets in train_loader:
        data = data.permute(1, 0, 2)
        utils.reset(net)
        spk_rec, mem_rec = net(data)
        logits = spk_rec.sum(dim=0)
        
        loss = loss_fn(logits, targets)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        bs = targets.size(0)
        total_loss += loss.item() * bs
        total_correct += (logits.argmax(dim=1) == targets).sum().item()
        total_n += bs
    print(f"Epoch{epoch+1}/{num_epochs} | Loss={total_loss/total_n:.4f} | Acc={total_correct/total_n:.3f}")


Epoch1/200 | Loss=0.0388 | Acc=0.982
Epoch2/200 | Loss=0.0186 | Acc=0.994
Epoch3/200 | Loss=0.0182 | Acc=0.994
Epoch4/200 | Loss=0.0177 | Acc=0.994
Epoch5/200 | Loss=0.0193 | Acc=0.993
Epoch6/200 | Loss=0.0170 | Acc=0.994
Epoch7/200 | Loss=0.0171 | Acc=0.994
Epoch8/200 | Loss=0.0168 | Acc=0.994
Epoch9/200 | Loss=0.0163 | Acc=0.995


KeyboardInterrupt: 